# Initialization

Notebook for the C-MAPPS benchmark. Plot the real vs the predicted RUL 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import model_from_json, clone_model

from data_handler_CMAPS import CMAPSDataHandler
import custom_scores
import CMAPSAuxFunctions
from tunable_model import SequenceTunableModelRegression
from keras import regularizers

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

import matplotlib.pyplot as plt

Using TensorFlow backend.


# Function to plot RULs

In [2]:
def plot_RUL(real_rul_list, predicted_rul_list):
    
    for real_rul, predicted_rul, i in zip(real_rul_list, predicted_rul_list, range(len(real_rul_list))):
        
        x = np.arange(1,real_rul.shape[0]+1)
        e = real_rul-predicted_rul
        
        plt.clf()
        fig = plt.figure(i)
        fig.suptitle('Real RUL vs Predicted RUL for FD00'+str(i+1), fontsize=14)
        #plt.title('Real RUL vs Predicted RUL')
        
        plt.subplot(2,1,1)
        plt.xlabel("Engine Number")
        plt.ylabel("RUL")
        plt.plot(x, predicted_rul, 'r--')
        plt.plot(x, real_rul, 'g--')
        plt.legend(('Predicted RUL', 'Real RUL'), loc='center left', bbox_to_anchor=(1, 0.5))
        
        y_ticks_array = gen_yticks(e)
        
        plt.subplot(2,1,2)
        plt.xlabel("Engine Number")
        plt.ylabel("Prediction Error")
        plt.plot(x, e, 'm--')
        plt.yticks(y_ticks_array)
        
        #plt.show()
        plt.savefig('rul_plots_dataset_'+str(i+1)+'.png', dpi=900, format='png', bbox_inches='tight')

In [3]:
def gen_yticks(error_array):
    
    min_val = np.amin(error_array)
    max_val = np.amax(error_array)
    min_val_yticks_multiplier = math.floor(min_val/25)
    max_val_yticks_multiplier =  math.ceil(max_val/25)
    min_val_yticks = 25*min_val_yticks_multiplier
    max_val_yticks = 25*max_val_yticks_multiplier
        
    y_ticks_array = np.arange(min_val_yticks, max_val_yticks+25, 25)
    
    if y_ticks_array.size > 6:
        min_val_yticks_multiplier = math.floor(min_val/50)
        max_val_yticks_multiplier =  math.ceil(max_val/50)
        min_val_yticks = 50*min_val_yticks_multiplier
        max_val_yticks = 50*max_val_yticks_multiplier
        
        y_ticks_array = np.arange(min_val_yticks, max_val_yticks+50, 50)
        
    return y_ticks_array

In [4]:
def gen_stats(real_rul_list, predicted_rul_list):
    
    for real_rul, predicted_rul, i in zip(real_rul_list, predicted_rul_list, range(len(real_rul_list))):
        
        e = real_rul-predicted_rul
        e_abs = np.absolute(e)
        e_abs.sort()
        
        first_percentile = np.percentile(e_abs, 25, interpolation='lower')
        second_percentile = np.percentile(e_abs, 50, interpolation='lower')
        third_percentile = np.percentile(e_abs, 75, interpolation='lower')
        
    
        print("For FD001"+str(i+1))
        #print(e)
        #print(e_abs)
        print(stats.describe(e))
        print(stats.describe(e_abs))
        print(first_percentile)
        print(second_percentile)
        print(third_percentile)

# Create data Handler

In [5]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Load models and get predicted and real RULs

In [6]:
datasets = [1, 2, 3, 4]

window_sizes = {1:24,2:17,3:24,4:17}
strides = {1:1,2:1,3:1,4:1}
max_ruls = {1:129, 2:139, 3:129, 4:139}

y_predicted_list = []
y_true_list = []


for i in datasets:

    # load json and create model
    json_file = open('bestRULModel_dataset_'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("bestRULModel_dataset_"+str(i)+".h5")
    print("Loaded model from disk")


    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]

    #Selected as per CNN paper
    selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                         'BPR', 'htBleed', 'W31', 'W32']

    #Create and compile the models
    #nFeatures = len(selected_features)
    #shapeSN = nFeatures*windowSize
    #modelRULSN = RULmodel_SN_1(shapeSN)
    loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
    min_max_scaler = MinMaxScaler()

    tModel = SequenceTunableModelRegression('ModelRUL_SN_1', loaded_model, lib_type='keras', data_handler=dHandler_cmaps)

    tModel.data_handler.change_dataset(i)
    tModel.data_handler.sequence_length = window_sizes[i]
    tModel.data_handler.sequence_stride = strides[i]
    tModel.data_handler.max_rul = max_ruls[i]

    tModel.data_handler.data_scaler = None
    tModel.data_scaler = min_max_scaler

    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    #tModel.print_data()
    #print(tModel.model.get_weights())
    tModel.get_model_description()
    
    # evaluate loaded model on test data
    tModel.evaluate_model(['rhs', 'rmse'], round=2)
    print("scores")

    #print(tModel.y_pred)

    cScores = tModel.scores
    print(cScores)
    rmse = math.sqrt(cScores['score_1'])
    rmse2 = cScores['rmse']
    rhs = cScores['rhs']
    time = tModel.train_time
    
    y_predicted = tModel.y_predicted_rounded.flatten()
    y_true = tModel.y_test.flatten()
    
    y_predicted_list.append(y_predicted)
    y_true_list.append(y_true)

    print(cScores)
    print("RMSE: {}".format(rmse))
    print("RMSE2: {}".format(rmse2))
    print("RHS: {}".format(rhs))
    print("Time : {} seconds".format(time))

Loaded model from disk
Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes
Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 20)                6740      
_________________________________________________________________
fc2 (Dense)                  (None, 20)                420       
_________________________________________________________________
out (Dense)                  (None, 1)                 21        
Total params: 7,181
Trainable params: 7,181
Non-trainable params: 0
_________________________________________________________________
100/100 [==============================] - 0s 1ms/step
scores
{'loss': 270.64216979980466, 'score_1': 206.3658889770508, 'rhs': 3.4385813531834817, 'rmse': 14.358272876638054}
{'loss': 270.6

In [7]:
plot_RUL(y_predicted_list, y_true_list)

In [8]:
gen_stats(y_predicted_list, y_true_list)

For FD0011
DescribeResult(nobs=100, minmax=(-41.0, 38.0), mean=0.52, variance=207.96929292929292, skewness=0.0809170551822932, kurtosis=0.7991521728626081)
DescribeResult(nobs=100, minmax=(0.0, 41.0), mean=10.12, variance=104.79353535353536, skewness=1.1960458423537905, kurtosis=0.38924759674704656)
2.0
6.0
14.0
For FD0012
DescribeResult(nobs=259, minmax=(-138.0, 83.0), mean=-1.776061776061776, variance=1090.360510011673, skewness=-0.5417818470354242, kurtosis=0.9387414026691236)
DescribeResult(nobs=259, minmax=(0.0, 138.0), mean=24.386100386100384, variance=496.5402711681782, skewness=1.3134015218145407, kurtosis=2.2202119564775806)
6.0
19.0
38.0
For FD0013
DescribeResult(nobs=100, minmax=(-36.0, 56.0), mean=1.41, variance=281.1938383838384, skewness=0.28982269899486174, kurtosis=0.4573256719072951)
DescribeResult(nobs=100, minmax=(0.0, 56.0), mean=12.87, variance=115.89202020202022, skewness=1.2088221192657573, kurtosis=1.6271374565790353)
5.0
10.0
18.0
For FD0014
DescribeResult(nobs

# Create plot for illustrating the data reshape

In [13]:
def plot_windows(plotting_data):
    
    plt.clf()
    
    x = range(0, plotting_data.shape[0])
    
    for i in range(plotting_data.shape[1]):
        
        y = plotting_data[:,i]
        y = y+i
        plt.plot(x, y)
        
    plt.plot([0,0], [0,14], linestyle='-.', color='k')
    plt.plot([5,5], [0,14], linestyle='-.', color='k')
    
    plt.plot([4,4], [0,14], linestyle='-.', color='r')
    plt.plot([9,9], [0,14], linestyle='-.', color='r')
        
    plt.ylabel("Sensor number")
    plt.xlabel("Engine cycle")
    plt.title("Window and stride")
    #plt.show()
    plt.savefig('window_stride.png', dpi=900, format='png', bbox_inches='tight')

In [14]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

min_max_scaler = MinMaxScaler()

dHandler_cmaps2 = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
dHandler_cmaps2.data_scaler = MinMaxScaler()
dHandler_cmaps2.load_data(verbose=1, unroll=False)
#dHandler_cmaps2.print_data()
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)


plotting_data = dHandler_cmaps2.X_train[0]
plot_windows(plotting_data)

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Loading data from file and computing dataframes


In [18]:
#Plot ReLu and Linear

x = np.linspace(-10,10, 100)
relu = x*(x>0)

plt.clf()
plt.plot(x, x, color='b')
plt.ylabel("linear(x)")
plt.xlabel("x")
plt.title("Linear activation function")
plt.savefig('linear_function.png', dpi=900, format='png', bbox_inches='tight')

plt.clf()
plt.plot(x, relu, color='b')
plt.ylabel("relu(x)")
plt.xlabel("x")
plt.title("ReLU activation function")
plt.savefig('relu_function.png', dpi=900, format='png', bbox_inches='tight')